In [1]:
import sys
sys.path.append('../')
import random
import numpy as np
import pandas as pd
from time import sleep
import numpy as np
import gym
import csv
from collections import defaultdict
import os
import argparse
import matplotlib.pyplot as plt
import maps

In [2]:
# Set OpenAI Environment
ENV_NAME = 'ed_grid:ed-grid-v0'
env = gym.make(ENV_NAME)
env.SetExplorationEnvironment()

# Set params
mode = 'test'#'train'
map_number = 4
output_filename = mode+'_map'+str(map_number)+'_config.csv'
num_high_acuity_patients = 5
num_low_acuity_patients = 5
num_configurations = 10
ed_dataset_df = 'ed_dataset_dataframe.csv'

# Initialization
high_acuity_videos = []
low_acuity_videos = []

In [3]:
# Read ED Dataset Dataframe
if(ed_dataset_df is not None):
    ed_dataset_df = pd.read_csv(ed_dataset_df)
#print(ed_dataset_df.head())

# Optical Flow 

In [4]:
high_acuity_of_df = ed_dataset_df[ed_dataset_df['low_high_acuity_of']==1]
low_acuity_of_df = ed_dataset_df[ed_dataset_df['low_high_acuity_of']==0]

In [5]:
# Configure Random Map Positions of teams 
def SetMapPositions(num_high_acuity, num_low_acuity,map_number):
    clin_row, clin_col, non_clin_row, non_clin_col = [], [], [], []
    for i in range(num_high_acuity):
        env.reset_env(map_num=map_number)
        for (row, col) in env.team_locs:
            #print(row,col)
            clin_row.append(row)
            clin_col.append(col)
            if(len(clin_row)>num_high_acuity):
                break
        if(len(clin_row)>=num_high_acuity):
                break
    for i in range(num_low_acuity):
        for (row, col) in env.other_team_locs:
            non_clin_row.append(row)
            non_clin_col.append(col)
            if(len(non_clin_row)>num_low_acuity):
                break
        if(len(non_clin_row)>=num_low_acuity):
                break
    return clin_row, clin_col, non_clin_row, non_clin_col

In [6]:
ha_row, ha_col, la_row, la_col = SetMapPositions(num_high_acuity_patients, num_low_acuity_patients,map_number)
print(ha_row, ha_col, la_row, la_col)
print(ha_row+la_row)

[7, 2, 8, 7, 9] [2, 0, 0, 7, 2] [6, 11, 3, 6, 9] [9, 2, 2, 10, 9]
[7, 2, 8, 7, 9, 6, 11, 3, 6, 9]


In [7]:
def get_acuity(optical_flow,max_num_people,num_high_acuity_patients,num_low_acuity_patients):
    acuity_score = []
    for of, num_people in zip(optical_flow,max_num_people):
        ac = of*(num_people/(1+num_high_acuity_patients+num_low_acuity_patients))
        acuity_score.append(ac)
    min_as_of=min(acuity_score)
    max_as_of=max(acuity_score)
    acuity_score = [((i-min_as_of)/(max_as_of-min_as_of)) for i in acuity_score]
    return acuity_score

In [8]:
video_name = []
segment_number = []
rows = []
cols = []
acuity_score = []
config_num = []
robot_row = []
robot_col = []
optical_flow = []
max_num_people = []

for i in range(num_configurations):
    optical_flow_local = []
    max_num_people_local = []
    
    # Generate Robot row-col
    env.GenerateRobotLocation()
    [(r_row,r_col)] = env.robot_loc
    
    # Patient rows and cols
    ha_row, ha_col, la_row, la_col = SetMapPositions(num_high_acuity_patients, num_low_acuity_patients,map_number)
    for r in ha_row+la_row:
        rows.append(r)
    for c in ha_col+la_col:
        cols.append(c)
        
    high_acuity_idx = [random.randint(0,len(high_acuity_of_df)-1) 
                                for i in range(num_high_acuity_patients)]
    low_acuity_idx = [random.randint(0,len(low_acuity_of_df)-1) 
                                for i in range(num_low_acuity_patients)]
    for h in high_acuity_idx:
        optical_flow_local.append(high_acuity_of_df.iloc[h]['optical_flow_norm'])
        max_num_people_local.append(high_acuity_of_df.iloc[h]['max_num_people'])
        segment_number.append(high_acuity_of_df.iloc[h]['segment_number'])        
        video_name.append(high_acuity_of_df.iloc[h]['video_names'])
        robot_row.append(r_row)
        robot_col.append(r_col)
        config_num.append(i)
        
    for l in low_acuity_idx:
        optical_flow_local.append(low_acuity_of_df.iloc[l]['optical_flow_norm'])
        max_num_people_local.append(low_acuity_of_df.iloc[l]['max_num_people'])
        segment_number.append(low_acuity_of_df.iloc[l]['segment_number'])        
        video_name.append(low_acuity_of_df.iloc[l]['video_names'])
        robot_row.append(r_row)
        robot_col.append(r_col)
        config_num.append(i)
    optical_flow.append([i for i in optical_flow_local])
    max_num_people.append([i for i in max_num_people_local])
    acuity_list = get_acuity(optical_flow_local,max_num_people_local,num_high_acuity_patients,num_low_acuity_patients)
    acuity_score.append([i for i in acuity_list])
    

optical_flow = [item for sublist in optical_flow for item in sublist]
max_num_people = [item for sublist in max_num_people for item in sublist]
acuity_score = [item for sublist in acuity_score for item in sublist]

In [9]:
print(len(video_name))
print(len(optical_flow))
print(len(max_num_people))
print(len(segment_number))
print(len(acuity_score))
print(len(robot_col))
print(len(robot_row))
print(len(config_num))
print(len(rows))
print(len(cols))

100
100
100
100
100
100
100
100
100
100


In [10]:
# Export data into pandas dataframe
data = {'video_name':video_name,
        'optical_flow':optical_flow,
        'max_num_people':max_num_people,
        'segment_number':segment_number,
        'acuity_score':acuity_score,
        'robot_col':robot_col,
        'robot_row':robot_row,
        'config_num':config_num,
        'row':rows,
        'col':cols}
 
# Create DataFrame
optical_flow_df = pd.DataFrame(data)
 
# Print the output.
#print(df)

# Save dataframe
if(not os.path.exists('./map_config/')):
    os.makedirs('./map_config/')
optical_flow_df.to_csv('./map_config/optical_flow_'+output_filename)
print('Saving map config to optical_flow_'+output_filename)

Saving map config to optical_flow_test_map4_config.csv
